In [120]:
import pandas as pd
import numpy as np

## Data Processing - Zulu dataset

In [99]:
#GOV-ZA.50000ParallelleEnWoorde.zu.pos.full
train_zu = pd.read_excel('zu/2.POS Annotated/GOV-ZA.50000ParallelleEnWoorde.zu.pos.full.xls')
test_zu = pd.read_excel('zu/2.POS Annotated/GOV-ZA.Toetsteks.5000ParallelleEnWoorde.zu.pos.full.xls')
train_zu.head()

,Token,POS
0,Injongo,N
1,ngezinkonzo,P
2,ze-website,V
3,yeNingizimu,N05
4,Afrika,N


In [100]:
test_zu.head(20)

,Token,POS
0,Thola,V
1,ifomu,N05
2,lesicelo,N07
3,kwi-inthanethi,COP
4,noma,CONJ
5,emahhovisi,N06
6,e-SARS,REL
7,aseduzane,V
8,nawe,ADV
9,.,PUNCT


In [101]:
train_zu = train_zu[~(train_zu['POS'] == 'PUNCT')]

In [102]:
train_zu[train_zu['POS'].isna() & ~train_zu['Token'].isna()]

,Token,POS


In [103]:
train_zu = train_zu.fillna("END")
train_zu.tail()

,Token,POS
44318,nenani,N05
44319,lemali,N09
44320,okumele,V
44321,uyikhokhe,V
44323,END,END


In [104]:
test_zu = test_zu[~(test_zu['POS'] == 'PUNCT')]
test_zu = test_zu.fillna("END")
test_zu.tail()

,Token,POS
4670,kube,V
4671,ngoyimpumelelo,N09
4672,ezweni,N05
4673,lethu,POS05
4675,END,END


In [117]:
len(set(test_zu.POS.unique())), len(set(train_zu.POS.unique()))

(77, 99)

In [82]:
pos_list = train_zu.POS.unique()
pos_list

array(['N', 'P', 'V', 'N05', 'N03', 'QUANT03', 'N10', 'N01', 'END', 'ADV',
       'N07', 'N04', 'N09', 'CONJ', 'QUANT08', 'PRON02', 'PRON08', 'N06',
       'LOC', 'POS', 'N08', 'N02', 'REL', 'DEM02', 'COP', 'ADJ04',
       'ADJ01', 'DEM01', 'POS15', 'ADJ00', 'REL08', 'N11', 'N00',
       'PRON06', 'PRON', 'PRON04', 'QUANT04', 'POS05', 'POS04', 'POS03',
       'POS11', 'N14', 'ADJ06', 'DEM', 'IDEOPH', 'ADJ07', 'M', 'PRON09',
       'POS08', 'ADJ09', 'POS09', 'REL09', 'PRON01', 'QUANT10', 'INT',
       'ADJ02', 'ADJ03', 'POS06', 'POS02', 'PRON05', 'POS07', 'REL11',
       'ADJ05', 'PRON15', 'QUANT06', 'QUANT02', 'ADJ08', 'ADJ', 'POS10',
       'REL10', 'QUANT11', 'QUANT15', 'REL06', 'NUM03', 'QUANT01',
       'QUANT09', 'POS01', 'PRON07', 'NUM06', 'REL15', 'PRON10',
       'QUANT07', 'REL05', 'REL02', 'PRON11', 'REL07', 'REL04', 'REL01',
       'NUM09', 'QUANT05', 'REL03', 'NUM15', 'PRON03', 'POS14', 'PRES',
       'QUANT14', 'NUM07', 'REL14', 'DEM03'], dtype=object)

## Part 1

##### Counting

In [168]:
tag_transitions = {}
for i in range(len(pos_list)):
    for j in range(len(pos_list)):
        key = (pos_list[i], pos_list[j])
        if key in tag_transitions:
            continue
        else:
            tag_transitions[key] = tag_transitions.setdefault(key, 0)

In [169]:
zu_numpy = train_zu.to_numpy()
#zu_pos_given_pos = dict()
zu_words_given_pos = dict()

for i in range(len(zu_numpy)):
    t, p = zu_numpy[i]
    if i == 0:  # Start of the DF, start of the verify first sentence
        zu_words_given_pos[(t, p)] = 1
        tag_transitions[(p, 'END')] = 1
        
    else:
        t_before, p_before = zu_numpy[i-1]
        #t_after, p_after = zu_numpy[i+1]
        
        if t != "END":
            zu_words_given_pos[(t, p)] = zu_words_given_pos.setdefault((t, p), 0) + 1
            
        tag_transitions[(p, p_before)] += 1
        

In [170]:
zu_words_given_pos

{('Injongo', 'N'): 2,
 ('ngezinkonzo', 'P'): 2,
 ('ze-website', 'V'): 2,
 ('yeNingizimu', 'N05'): 6,
 ('Afrika', 'N'): 128,
 ('wukutholakala', 'V'): 1,
 ('komthombo', 'N03'): 1,
 ('owodwa', 'QUANT03'): 5,
 ('wolwazi', 'V'): 3,
 ('ngezinkonzo', 'N10'): 6,
 ('eziphakelwa', 'V'): 1,
 ('nguhulumeni', 'N01'): 5,
 ('waseNingizimu', 'N05'): 7,
 ('wukuphungula', 'V'): 1,
 ('igebe', 'N05'): 1,
 ('elikhona', 'ADV'): 1,
 ('phakathi', 'ADV'): 71,
 ('kwabanamandla', 'P'): 1,
 ('kanye', 'ADV'): 506,
 ('nabangenamandla', 'P'): 1,
 ('kwabomnotho', 'P'): 1,
 ('womkhakha', 'N03'): 2,
 ('wokuqala', 'V'): 5,
 ('nabomnotho', 'P'): 1,
 ('wesibili', 'N07'): 2,
 ('I-website', 'N'): 3,
 ('iyimizamo', 'N04'): 1,
 ('ebalulekile', 'V'): 9,
 ('yokungenela', 'V'): 1,
 ('nokusiza', 'V'): 3,
 ('ngenhloso', 'N09'): 18,
 ('yokuqinisekisa', 'V'): 1,
 ('ukuthi', 'CONJ'): 479,
 ('zonke', 'QUANT08'): 34,
 ('izakhamizi', 'V'): 3,
 ('zikwazi', 'V'): 1,
 ('ukuxhumana', 'V'): 11,
 ('nohulumeni', 'N01'): 3,
 ('Abasebenzisi', 'V

##### Smoothing

##### Smoothing the tags

In [171]:
for k, v in tag_transitions.items():
    tag_transitions[k] += 1

In [172]:
for tag in pos_list:
    count_tags = 0
    for k, v in tag_transitions.items():
        if k[1] == tag:
            count_tags += v
            
    for k, v in tag_transitions.items():
        if k[1] == tag:
            tag_transitions[k] /= count_tags

In [173]:
# Testing smoothing
count_tags = 0
for k, v in tag_transitions.items():
    if k[1] == 'V':
        count_tags += v
        
count_tags

0.9999999999999999

##### Smoothing the emission count

In [154]:
train_vocab = list(set(train_zu['Token']))
test_vocab = list(set(test_zu['Token']))
all_vocabs = list(set(train_vocab + test_vocab)); len(all_vocabs)

15224

In [165]:
all_zu_words_given_pos = zu_words_given_pos.copy()

for t in all_vocabs:
    for p in pos_list:
        all_zu_words_given_pos[(t, p)] = all_zu_words_given_pos.setdefault((t, p), 0) + 0

In [166]:
len(all_zu_words_given_pos), len(zu_pos_given_pos)

(1507176, 2464)

In [174]:
for k, v in all_zu_words_given_pos.items():
    all_zu_words_given_pos[k] += 1

In [175]:
for tag in pos_list:
    count_tags = 0
    for k, v in all_zu_words_given_pos.items():
        if k[1] == tag:
            count_tags += v
            
    for k, v in all_zu_words_given_pos.items():
        if k[1] == tag:
            all_zu_words_given_pos[k] /= count_tags

In [176]:
all_zu_words_given_pos

{('Injongo', 'N'): 0.00017297047970479704,
 ('ngezinkonzo', 'P'): 0.00017839091395611584,
 ('ze-website', 'V'): 0.00011070110701107011,
 ('yeNingizimu', 'N05'): 0.000426725188978298,
 ('Afrika', 'N'): 0.007437730627306273,
 ('wukutholakala', 'V'): 7.380073800738008e-05,
 ('komthombo', 'N03'): 0.00012239902080783354,
 ('owodwa', 'QUANT03'): 0.0003935458480913026,
 ('wolwazi', 'V'): 0.00014760147601476016,
 ('ngezinkonzo', 'N10'): 0.0004337588300904697,
 ('eziphakelwa', 'V'): 7.380073800738008e-05,
 ('nguhulumeni', 'N01'): 0.00037539886129012075,
 ('waseNingizimu', 'N05'): 0.000487685930260912,
 ('wukuphungula', 'V'): 7.380073800738008e-05,
 ('igebe', 'N05'): 0.000121921482565228,
 ('elikhona', 'ADV'): 0.00011414873580275098,
 ('phakathi', 'ADV'): 0.004109354488899036,
 ('kwabanamandla', 'P'): 0.00011892727597074389,
 ('kanye', 'ADV'): 0.028936704525997375,
 ('nabangenamandla', 'P'): 0.00011892727597074389,
 ('kwabomnotho', 'P'): 0.00011892727597074389,
 ('womkhakha', 'N03'): 0.000183598

### Viterbi

In [189]:
A = np.zeros((len(pos_list), len(pos_list)))
for i in range(len(pos_list)):   # z_t
    for j in range(len(pos_list)):  # z_{t-1}
        A[i][j] = tag_transitions[(pos_list[i], pos_list[j])]

In [191]:
A

array([[1.14465976e-01, 9.81087470e-02, 4.17536534e-02, ...,
        1.00000000e-02, 9.90099010e-03, 9.90099010e-03],
       [5.63316809e-02, 4.55082742e-02, 5.12734864e-02, ...,
        1.00000000e-02, 9.90099010e-03, 9.90099010e-03],
       [2.79405137e-01, 2.45271868e-01, 2.61461378e-01, ...,
        2.00000000e-02, 2.97029703e-02, 9.90099010e-03],
       ...,
       [4.50653447e-04, 5.91016548e-04, 8.35073069e-05, ...,
        1.00000000e-02, 9.90099010e-03, 9.90099010e-03],
       [4.50653447e-04, 5.91016548e-04, 8.35073069e-05, ...,
        1.00000000e-02, 9.90099010e-03, 9.90099010e-03],
       [4.50653447e-04, 5.91016548e-04, 8.35073069e-05, ...,
        1.00000000e-02, 9.90099010e-03, 9.90099010e-03]])

In [270]:
def viterbi(sentence):
    delta = []
    psi = []
    
    ## Initialization
    word = sentence[0]
    delta_w = []
    for tag in pos_list:
        d = all_zu_words_given_pos[(word, tag)] * tag_transitions[(tag, 'END')]
        delta_w.append(d)
            
    delta.append(np.asarray(delta_w))
    
    ## Recursion
    for i in range(1, len(sentence)):
        word = sentence[i]
        delta_w = []
        psi_w = []
        for j in range(len(pos_list)):
            tag = pos_list[j]
            d = all_zu_words_given_pos[(word, tag)] * np.max(A[:, j] * delta[-1])
            ps = np.argmax(all_zu_words_given_pos[(word, tag)] * A[:, j] * delta[-1])
            
            delta_w.append(d)
            psi_w.append(int(ps))
            
        delta.append(delta_w)
        psi.append(psi_w)
        
    ## Termination
    z_T = []
    for i in range(len(pos_list)):
        z_T.append(tag_transitions[("END", pos_list[i])] * delta[-1][i])
        
    z_T = np.argmax(z_T)
    
    ## Backtracking
    z_hat = [z_T]
    for i in range(len(psi)):
        t = len(psi) - i - 1
        z_hat.append(psi[t][z_hat[-1]])
        
    z_hat.reverse()
    
    return delta, z_hat

In [221]:
sentences = []
sentence = []
for k in range(len(test_zu_numpy)):
    tok = test_zu_numpy[k][0]
    if tok == "END":
        sentences.append(sentence)
        sentence = []
    else:
        sentence.append(tok)

In [271]:
d, z = viterbi(sentences[1])
z

[2, 10, 2, 2, 2, 10, 9, 2, 7, 10, 2, 2, 9, 2, 10, 13, 19]

In [280]:
predicted_tags = []
correct = 0
total = 0
for sentence in sentences:
    d, z = viterbi(sentence)
    predicted_tags += z

In [281]:
for test in test_zu_numpy:
    token, pos = test
    if pos == "END":
        continue

    if pos_list[predicted_tags[total]] == pos:
        correct += 1
    
    total += 1

In [285]:
print("The model's accuracy:", str(correct/total*100)+"%")

The model's accuracy: 60.102370689655174%
